In [ ]:
# Review 2
""""""""""""""""""
import cv2
from ultralytics import YOLO
import requests

# Load YOLO model
model = YOLO("../models/yantra_failure.pt")
raspberry_pi_ip = "http://192.168.174.105:8000"

car_length = 7  
road_length = 24  

def test_traffic_light(endpoint):
    url = f"{raspberry_pi_ip}/{endpoint}"
    response = requests.get(url)
    print(f"Testing {endpoint}: {response.json()}")

test_traffic_light("red1_red2")  # Set initial state
flag = "r1r2"

cap = cv2.VideoCapture("http://192.168.174.136:8080/video")

line_y = 790  
line_x1 = 495
line_x2 = 1558
radius = 20

straight_counter = 0
left_counter = 0
right_counter = 0
traffic_density = 0
prev_traffic_density = 0  

frame_skip = 3  
frame_count = 0

# Track when last API request was made to avoid spamming Raspberry Pi
last_api_call = {"green2": 0, "green1_red2": 0, "green1_green2": 0}
api_cooldown = 30  # Number of frames to wait before sending the same API call again

# Track frames without vehicles
no_vehicle_counter = 0  
no_vehicle_threshold = 30  

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video")
        break

    if frame_count % frame_skip != 0:
        frame_count += 1
        continue  

    results = model(frame)  
    vip_detected = False  
    vehicle_detected = False  

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            conf = box.conf[0].item()

            if conf >= 0.60:
                cls = int(box.cls[0])
                label = f"{model.names[cls]} {conf:.2f}"
                label1 = model.names[cls]

                vehicle_detected = True  

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.circle(frame, (center_x, center_y), radius, (0, 0, 255), -1)

                if center_y + radius > line_y and center_y - radius < line_y:
                    straight_counter += 1
                
                traffic_density = (car_length * straight_counter) / (2 * road_length)

                # Only trigger if density actually changes and cooldown has passed
                if traffic_density >= 0.4 and flag != "g2" and frame_count - last_api_call["green2"] > api_cooldown:
                    test_traffic_light("green2")
                    last_api_call["green2"] = frame_count  
                    flag = "g2"
                    if traffic_density>=0.3:
                        
                        traffic_density-=0.3
                    

                if center_x + radius > line_x2 and center_x - radius < line_x2:
                    left_counter += 1
                if center_x + radius > line_x1 and center_x - radius < line_x1:
                    right_counter += 1

                if label1 == "vip":
                    vip_detected = True  
                    if flag != "g1g2" and frame_count - last_api_call["green1_green2"] > api_cooldown:
                        test_traffic_light("green1_green2")
                        last_api_call["green1_green2"] = frame_count  
                        flag = "g1g2"
                    break  

    if not vip_detected:  
        for result in results:
            for box in result.boxes:
                conf = box.conf[0].item()
                if conf >= 0.60:
                    cls = int(box.cls[0])
                    label1 = model.names[cls]
    ######## Code added here flag!="g2"
                    if label1 == "cars" and flag != "g1r2" and flag!="g2" and frame_count - last_api_call["green1_red2"] > api_cooldown:
                        test_traffic_light("green1_red2")
                        last_api_call["green1_red2"] = frame_count  
                        flag = "g1r2"
                        break  

    # If no vehicles detected, increase no_vehicle_counter
    if not vehicle_detected:
        no_vehicle_counter += 1
    else:
        no_vehicle_counter = 0  

    # If no vehicles detected for too long, turn red1_red2
    if no_vehicle_counter >= no_vehicle_threshold and flag != "r1r2":
        test_traffic_light("red1_red2")
        flag = "r1r2"
        no_vehicle_counter = 0  

    cv2.line(frame, (1689, line_y), (203, line_y), (0, 255, 0), 3)
    cv2.line(frame, (line_x2, 1000), (line_x2, 100), (0, 255, 0), 3)
    cv2.line(frame, (line_x1, 1000), (line_x1, 100), (0, 255, 0), 3)

    cv2.putText(frame, f"S Counter: {straight_counter}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"L Counter: {left_counter}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"R Counter: {right_counter}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Traffic Density: {traffic_density:.2f}", (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Video with Line", frame)
    frame_count += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        test_traffic_light("turn_off")
        break

cap.release()
cv2.destroyAllWindows()


Testing red1_red2: {'status': 'Red1 ON, Red2 ON'}

0: 384x640 (no detections), 54.4ms
Speed: 4.0ms preprocess, 54.4ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 36.9ms
Speed: 1.9ms preprocess, 36.9ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 46.6ms
Speed: 1.3ms preprocess, 46.6ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 42.5ms
Speed: 1.3ms preprocess, 42.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 74.7ms
Speed: 1.7ms preprocess, 74.7ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 40.0ms
Speed: 1.6ms preprocess, 40.0ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 42.8ms
Speed: 1.6ms preprocess, 42.8ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no det

2025-02-09 10:21:21.576 python[16977:3110931] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-09 10:21:21.576 python[16977:3110931] +[IMKInputSession subclass]: chose IMKInputSession_Modern


0: 384x640 (no detections), 40.9ms
Speed: 1.7ms preprocess, 40.9ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 43.2ms
Speed: 1.8ms preprocess, 43.2ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 42.4ms
Speed: 1.3ms preprocess, 42.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.0ms
Speed: 1.3ms preprocess, 38.0ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 45.4ms
Speed: 1.4ms preprocess, 45.4ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 51.1ms
Speed: 1.5ms preprocess, 51.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 44.8ms
Speed: 1.5ms preprocess, 44.8ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 42.4ms
Speed: 1.4ms preprocess, 42.4ms in

[mjpeg @ 0x15b36b0e0] overread 8


0: 384x640 2 carss, 42.9ms
Speed: 1.7ms preprocess, 42.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 carss, 37.5ms
Speed: 1.4ms preprocess, 37.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 carss, 38.4ms
Speed: 2.3ms preprocess, 38.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 carss, 38.4ms
Speed: 1.4ms preprocess, 38.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cars, 32.2ms
Speed: 1.6ms preprocess, 32.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cars, 32.6ms
Speed: 1.3ms preprocess, 32.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cars, 34.9ms
Speed: 1.6ms preprocess, 34.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 cars, 37.2ms
Speed: 1.6ms preprocess, 37.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 3